In [ ]:
%matplotlib widget
from src.config import DATA_DIR
import h5py
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import matplotlib.pyplot as plt
import os
import glob
import mediapy as media
import cv2

In [ ]:
f = h5py.File("/home/jannik/Repos/demonstration-interface/data/session_20240620_121229/episode_20240620_121243.h5",'r')
for key in f.keys():
    print(f"{f[key].dtype}\t {key}  {f[key].shape}")

In [ ]:
timestamps = np.array(f["timestamps"])

color_images = np.array(f['color_images'])
color_images = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in color_images])

depth_images = np.array(f['depth_images'])
depth_images = cv2.normalize(depth_images, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_32F)

trigger_states = np.array(f['trigger_states'])
gripper_states = np.array(f['gripper_states'])

poses = np.array(f['pose_values'])

In [ ]:
fig = plt.figure()
plt.plot(range(len(trigger_states)), trigger_states)
plt.plot(range(len(gripper_states)), gripper_states)
plt.show()

In [ ]:
media.show_video(color_images, fps=30)

In [ ]:
media.show_video(depth_images, fps=30)

In [ ]:
def extract_translation(pose_matrix):
    return pose_matrix[:3, 3]

def extract_orientation(pose_matrix):
    return pose_matrix[:3, :3]

def transform_pose(pose, transformation):
    return np.dot(transformation, pose)

def set_axes_equal(ax):
    """
    Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    """

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [ ]:
T_x = np.array(
    [
        [1, 0, 0, 0],
        [0, -np.sqrt(2) / 2, np.sqrt(2) / 2, 0],
        [0, -np.sqrt(2) / 2, -np.sqrt(2) / 2, 0],
        [0, 0, 0, 1],
    ]
)

T_y = np.array([[0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0], [0, 0, 0, 1]])
print(np.dot(T_y, T_x))

In [ ]:
# ee_poses = np.array([transform_pose(np.dot(T_x,T_y),pose) for pose in poses])

translations = np.array([extract_translation(pose) for pose in poses])
orientations = np.array([extract_orientation(pose) for pose in poses])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')    
ax.set_box_aspect([1.0, 1.0, 1.0])
    
# Plot trajectory
ax.plot(translations[:, 0], translations[:, 1], translations[:, 2], '-', c="lightgrey", label='Trajectory')

translations = translations[::3]
orientations = orientations[::3]

# Plot orientation arrows
for i in range(len(translations)):
    t = translations[i]
    R = orientations[i]
    # R = np.dot(R_z, orientations[i])
    ax.quiver(t[0], t[1], t[2], R[0, 0], R[1, 0], R[2, 0], length=0.04, color='r', arrow_length_ratio=0)
    ax.quiver(t[0], t[1], t[2], R[0, 1], R[1, 1], R[2, 1], length=0.04, color='g', arrow_length_ratio=0)
    ax.quiver(t[0], t[1], t[2], R[0, 2], R[1, 2], R[2, 2], length=0.04, color='b', arrow_length_ratio=0)
    
# ax.quiver(0,0,0, 1, 0, 0, length=200, color='r', arrow_length_ratio=0)
# ax.quiver(0,0,0, 0,1,0, length=200, color='g', arrow_length_ratio=0)
# ax.quiver(0,0,0, 0,0,1, length=200, color='b', arrow_length_ratio=0)
    
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
    
ax.grid(True)
ax.view_init(elev=-70, azim=-45, roll=-45)

set_axes_equal(ax)
plt.show()